<a href="https://colab.research.google.com/github/statisticianinstilettos/Hackathon-Assets/blob/main/CrewAI%20MultiAgent%20Example/Opik_and_CrewAi_Agent_Observability_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Opik with CrewAI

This notebook showcases how to use Opik with CrewAI to build a simple Agent Architecture and implement AI Observability best practices.

<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>


[Opik](https://github.com/comet-ml/opik) is an open-source end-to-end model evaluation platform for AI developers. It offers best-in-class LLM evaluations, experiment tracking, and production monitoring. Think of it as a tool for keeping your AI Agent in check and auditing its decision-making.

Comet provides a hosted version of the Opik platform for free, simply create an account and grab your API Key. https://www.comet.com/site/products/opik/


[CrewAI](https://github.com/crewAIInc/crewAI) is a cutting-edge framework for orchestrating autonomous AI agents. CrewAI enables you to create AI teams where each agent has specific roles, tools, and goals, working together to accomplish complex tasks. Think of it as assembling your dream team - each member (Agent) brings unique skills and expertise, collaborating seamlessly to achieve your objectives.

You will need an [OpenAI](https://platform.openai.com/docs/api-reference/introduction)  API key as we will use OpenAI as our LLM provider for this project. You can find your API key here: https://platform.openai.com/api-keys.

Note: If you do not already have an OpenAI account, you can create get free access to a few models through OpenRouter following the prompt in the slides.


In [ ]:
%pip install crewai crewai-tools opik --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.3/573.3 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.8/467.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.5 MB/s e

## Preparing our environment

First, we set up our API keys for Opik and for our LLM-provider as environment variables:

In [ ]:
#Import all the packages we will be using
import opik
import os
import getpass
from crewai import Agent, Crew, Task, Process
from opik.integrations.crewai import track_crewai

In [ ]:
#Configure the Opik API key
opik.configure(use_local=False)

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "statisticianinstilettos" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [ ]:
### Use this cell if you are using your own OpenAI API key ###

#configure the OpenAI API key
#if "OPENAI_API_KEY" not in os.environ:
#    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
### Use this cell if you are using the OpenAI model access from OpenRouter ###

os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
#configure the OpenAI API key
if "OPENROUTER_API_KEY" not in os.environ:
    os.environ["OPENROUTER_API_KEY"] = getpass.getpass("Enter your OpenRouter API key: ")

Enter your OpenRouter API key: ··········


In [ ]:
os.environ["OPENAI_MODEL_NAME"] = "openrouter/google/gemini-2.0-flash-exp:free"

## Using CrewAI
The first step is to create our project. We will use an example from CrewAI's documentation:

In [ ]:
class YourCrewName:
    def agent_one(self) -> Agent:
        return Agent(
            role="Data Analyst",
            goal="Analyze data trends in the market",
            backstory="An experienced data analyst with a background in economics",
            verbose=True,
        )

    def agent_two(self) -> Agent:
        return Agent(
            role="Market Researcher",
            goal="Gather information on market dynamics",
            backstory="A diligent researcher with a keen eye for detail",
            verbose=True,
        )

    def task_one(self) -> Task:
        return Task(
            name="Collect Data Task",
            description="Collect recent market data and identify trends.",
            expected_output="A report summarizing key trends in the market.",
            agent=self.agent_one(),
        )

    def task_two(self) -> Task:
        return Task(
            name="Market Research Task",
            description="Research factors affecting market dynamics.",
            expected_output="An analysis of factors influencing the market.",
            agent=self.agent_two(),
        )

    def crew(self) -> Crew:
        return Crew(
            agents=[self.agent_one(), self.agent_two()],
            tasks=[self.task_one(), self.task_two()],
            process=Process.sequential,
            verbose=True,
        )

Now we can import Opik's tracker and run our `crew`:

In [ ]:
track_crewai(project_name="Opik-CrewAI-Agent-Observability-Workshop")

my_crew = YourCrewName().crew()
result = my_crew.kickoff()

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6cd71222-e6b4-4770-9d09-69cae1098c38                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8b842381-0b2e-4545-926b-9db2436091bb
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8b842381-0b2e-4545-926b-9db2436091bb
       Status: Executing Task...
    └── 🤖 Agent: Data Analyst
            Status: In Progress

# Agent: Data Analyst
## Task: Collect recent market data and identify trends.


🤖 Agent: Data Analyst
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 8b842381-0b2e-4545-926b-9db2436091bb
│      Assigned to: Data Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Data Analyst
│           Status: ✅ Completed
└── 📋 Task: 9ae8d5c1-1530-47e9-bd14-6d7d1046d3e9
       Assigned to: Market Researcher
       Status: ✅ Completed
    └── 🤖 Agent: Market Researcher
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9ae8d5c1-1530-47e9-bd14-6d7d1046d3e9                                                                     │
│  Agent: Market Researcher                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6cd71222-e6b4-4770-9d09-69cae1098c38                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

OPIK: Started logging traces to the "Opik-CrewAI-Agent-Observability-Workshop" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01966021-715e-77bb-a285-211d920d9c8b&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


**

**Factors Affecting Market Dynamics**

The market dynamics are influenced by a complex interplay of macroeconomic factors, industry-specific trends, and global events. Analyzing the provided Market Trends Report reveals several key factors:

**1. Macroeconomic Factors:**

*   **Inflation:** Elevated inflation rates are a primary driver of market dynamics. Persistently high inflation erodes consumer purchasing power, leading to reduced spending on discretionary items. Businesses face increased input costs, potentially squeezing profit margins. The report indicates a moderation in inflation in some sectors, but the overall level remains a significant concern.
*   **Interest Rates:** Central bank policies, particularly interest rate adjustments by the Federal Reserve, have a profound impact. Rising interest rates increase borrowing costs for businesses and consumers, dampening investment and spending. This can lead to a slowdown in economic growth and potentially trigger a recession. 

You can now go to the Opik app to see the trace:

![CrewAI trace in Opik](https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/fern/img/cookbook/crewai_trace_cookbook.png)

Create a Hallucination Eval Metric in the Comet UI.

Step 1. Navigate to the “Online Evaluation” tab and click “Create your first rule” \
Step 2. Configure your AI Provider \
Step 3. Select "Custom LLM as a Judge" \
Step 4. Copy and paste this prompt below into the open text field \
Step 5. Select the output to evaluate as `output.tasks_output[1].summary` \
Step 6. Save it and log another trace! \



**Hallucination Prompt:**

*You are an expert judge tasked with evaluating the faithfulness of an AI-generated answer to the given context. Analyze the provided OUTPUT to determine if the OUTPUT contains any hallucinations or unfaithful information.*

*Guidelines:*

*1. The OUTPUT should not contradict well-established facts or general knowledge. \
*2. Consider partial hallucinations where some information is correct but other parts are not.* \
*3. Pay close attention to the subject of statements. Ensure that attributes, actions, *or dates are correctly associated with the right entities (e.g., a person vs. a TV show they star in).* \
*4. Be vigilant for subtle misattributions or conflations of information, even if the date or other details are correct. 5. Check that the OUTPUT doesn't oversimplify or generalize information in a way that changes its meaning or accuracy.*

*Analyze the text thoroughly and assign a hallucination score between 0 and 1, where: - 0.0: The OUTPUT is entirely made up - 1.0: The OUTPUT is entirely factual*

*OUTPUT: {{output}}*
